In [23]:
from underthesea import sent_tokenize
import os
from tqdm import tqdm

def split_text_into_chunks(text, chunk_size=256, window_size=20):
    """Split a long text into multiple chunks (passages) with managable sizes.
    
    Args:
        chunk_size (int): Maximum size of a chunk.
        window_size (int): Decide how many words are overlapped between two consecutive chunks. Basically #overlapped_words = chunk_size - window_size.
    Returns:
        str: Multiple chunks of text splitted from initial document text.
    """
    words = text.split()
    num_words = len(words)
    chunks = []
    start_idx = 0

    while True:
        end_idx = start_idx + chunk_size
        chunk = " ".join(words[start_idx:end_idx])
        chunks.append(chunk)
        if end_idx >= num_words:
            break
        start_idx += window_size

    return chunks

def get_corpus(data_dir="data/data_raw10k/"):
    """Transform a corpus of documents into a corpus of passages.
    
    Args:
        data_dir (str): directory that contains .txt files, each file contains text content of a wikipedia page.
    Returns:
        str: A corpus of chunks splitted from multiple initial documents. Each chunk will contain information about (id, title, passage)
    """
    corpus = []
    meta_corpus = []
    # data_dir = "data/data_raw10k/"
    filenames = os.listdir(data_dir)
    filenames = sorted(filenames)
    print(len(filenames))
    _id = 0
    docs = {}
    for filename in tqdm(filenames):
        filepath = data_dir + filename
        title = filename.strip(".md")
        with open(filepath, "r") as f:
            text = f.read()
            docs[title] = text
            text = text.lstrip(title).strip()

            # No overlap.
            chunks = split_text_into_chunks(text, chunk_size=256, window_size=50)
            chunks = [f"{chunk}" for chunk in chunks]
            meta_chunks = [{
                "title": title,
                "passage": chunks[i],
                "id": _id + i,
                "len": len(chunks[i].split())
            } for i in range(len(chunks))]
            _id += len(chunks)
            corpus.extend(chunks)
            meta_corpus.extend(meta_chunks)
    return meta_corpus

In [24]:
meta_corpus = get_corpus(data_dir="E:/TrungPhanADVN/Code/advn_project/langchain_rag/app/api/data/training_data_2/")
print(f">>> Corpus size: {len(meta_corpus)}")
print(f">>> Example passage")
meta_corpus

52


100%|██████████| 52/52 [00:00<00:00, 3713.54it/s]

>>> Corpus size: 248
>>> Example passage


[{'title': '2nong_introduction',
  'passage': 'Ứng dụng 2Nông kết nối nông dân với các đơn vị thu mua nông sản Ứng dụng khoa học công nghệ số trong nông nghiệp đang dần mở ra cơ hội hiện đại hóa nền sản xuất, nhờ sở hữu nhiều ưu thế vượt trội so với nông nghiệp truyền thống. Trong vài năm gần đây, sự xuất hiện của các ứng dụng kết nối nông nghiệp đã mang đến giải pháp toàn diện cho nhiều vấn đề như cập nhật giá cả vật tư nông nghiệp, hỗ trợ canh tác, kết nối đầu ra cho nông sản... Từ đó, giải quyết bài toán nông nghiệp thời 4.0. App nông nghiệp - Giải pháp canh tác thông minh Việt Nam là quốc gia có nhiều lợi thế đặc biệt trong sản xuất nông nghiệp. Tuy nhiên, sự bùng nổ của công nghệ 4.0, quá trình hội nhập quốc tế yêu cầu chất lượng nông sản cao, đất canh tác bị thu hẹp do đô thị hóa... là một trong những thách thức lớn đối với nền nông nghiệp nước nhà. Trước tình hình hiện tại, sự ra đời của các ứng dụng công nghệ được đánh giá là giải pháp canh tác thông minh, nhằm nâng cao hiệu qu

In [25]:
meta_corpus

[{'title': '2nong_introduction',
  'passage': 'Ứng dụng 2Nông kết nối nông dân với các đơn vị thu mua nông sản Ứng dụng khoa học công nghệ số trong nông nghiệp đang dần mở ra cơ hội hiện đại hóa nền sản xuất, nhờ sở hữu nhiều ưu thế vượt trội so với nông nghiệp truyền thống. Trong vài năm gần đây, sự xuất hiện của các ứng dụng kết nối nông nghiệp đã mang đến giải pháp toàn diện cho nhiều vấn đề như cập nhật giá cả vật tư nông nghiệp, hỗ trợ canh tác, kết nối đầu ra cho nông sản... Từ đó, giải quyết bài toán nông nghiệp thời 4.0. App nông nghiệp - Giải pháp canh tác thông minh Việt Nam là quốc gia có nhiều lợi thế đặc biệt trong sản xuất nông nghiệp. Tuy nhiên, sự bùng nổ của công nghệ 4.0, quá trình hội nhập quốc tế yêu cầu chất lượng nông sản cao, đất canh tác bị thu hẹp do đô thị hóa... là một trong những thách thức lớn đối với nền nông nghiệp nước nhà. Trước tình hình hiện tại, sự ra đời của các ứng dụng công nghệ được đánh giá là giải pháp canh tác thông minh, nhằm nâng cao hiệu qu

In [26]:
import json
with open("data/corpus_chunks.jsonl", "w") as outfile:
    for chunk in meta_corpus:
        d = json.dumps(chunk, ensure_ascii=False) + "\n"
        outfile.write(d)

In [4]:
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize
from tqdm import tqdm 
import numpy as np
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder').cuda()
segmented_corpus = [tokenize(example["passage"]) for example in tqdm(meta_corpus)]
embeddings = model.encode(segmented_corpus)
embeddings /= np.linalg.norm(embeddings, axis=1)[:, np.newaxis]

100%|██████████| 48532/48532 [01:33<00:00, 517.08it/s]


In [5]:
import pickle
with open('data/corpus_embedding_w150.pkl', 'wb') as f:
    pickle.dump(embeddings, f)